In [ ]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.notebook import tqdm
from imaris_ims_file_reader.ims import ims
import dask.array as da
import pandas as pd
import numpy as np
import pickle as pkl
import napari
from cellpose import models
from skimage.io import imread,imsave
import seaborn as sns
import matplotlib.pyplot as plt

from utils import extract_regionprops, find_edge_df, mask_from_df, suppress_by_iou, compute_iou_array, add_soma_data, mark_tile_edge_objects

## Get an image

In [2]:
im_path = r'D:\data_analysis\2025_Birder_mito\88EM87C 25x25_ashlar.ome.tif'
output_dir = r'D:\data_analysis\2025_Birder_mito\C_00_analysis'
prefix_save = '88EM87C_'

axon_res = 3

px_size = 10 # in nm on res level 0
scale = ((px_size/1000)*2**axon_res, (px_size/1000)*2**axon_res)

In [3]:
store = imread(im_path, aszarr=True)
im = da.from_zarr(store, axon_res)
im_shape = im.shape

C:\Users\KMK280\AppData\Local\Temp\ipykernel_30168\465329857.py:1: FutureWarning: The plugin infrastructure in `skimage.io` is deprecated since version 0.25 and will be removed in 0.27 (or later). To avoid this warning, please do not pass additional keyword arguments for plugins (`**plugin_args`). Instead, use `imageio` or other I/O packages directly. See also `skimage.io.imread`.
  store = imread(im_path, aszarr=True)


In [4]:
viewer = napari.Viewer()
viewer.add_image(im, scale=scale)

<Image layer 'im' at 0x18b5a36bc50>

In [134]:
viewer.scale_bar.visible = True

# Text options
viewer.scale_bar.unit = 'um'  # set to None to diplay no unit
viewer.scale_bar.length = 20  # length, in units, of the scale bar
viewer.scale_bar.font_size = 30  # default is 10

# Text color
viewer.scale_bar.colored = True  # default value is False
viewer.scale_bar.color = 'white'  # default value is magenta: (1,0,1,1)

# Background box
viewer.scale_bar.box = False  # add background box, default is False

# Scale bar position
viewer.scale_bar.position = 'bottom_right'  # default is 'bottom_right'

## Clean SAM masks

In [5]:
properties = ['area', 'area_convex', 'area_filled', 'euler_number', 'image','bbox',
        'eccentricity', 'solidity', 'centroid', 'major_axis_length', 'minor_axis_length']


file_list = [x for x in os.listdir(output_dir) if x.startswith('tile_masks') and x.endswith('.pkl')]

df_all_list = []
for file_name in tqdm(file_list):

    print(f'Processing {file_name}')    

    df_path = os.path.join(output_dir,file_name)
    df = pd.read_pickle(df_path)
    print(f'Loaded {len(df)} masks.')

    # divide masks into separate objects
    props_list = [] 
    for ind, row in df.iterrows():
        props = extract_regionprops(row,properties,small_size=50)
        # keep origin of the data
        props['origin'] = ind
        props_list.append(props)

    props_all = pd.concat(props_list, ignore_index=True)
    props_all.columns = [ f'sc_{col}' if col == 'area' else col for col in props_all.columns]

    # drop the segmentation column
    df.drop(columns=['segmentation'], inplace=True)

    # Concatenate with the original DataFrame
    df['origin'] = df.index
    df_all = pd.merge(df, props_all,left_on = 'origin', right_on= 'origin', how = 'right').reset_index()

    df_all['label'] = df_all.index + 1

    print(f'Kept {len(df_all)} axons.')

    df_all_list.append(df_all)

  0%|          | 0/20 [00:00<?, ?it/s]

Processing tile_masks_00000_00000.pkl
Loaded 231 masks.


  5%|▌         | 1/20 [00:02<00:45,  2.37s/it]

Kept 243 axons.
Processing tile_masks_00000_00724.pkl
Loaded 396 masks.


 10%|█         | 2/20 [00:06<00:59,  3.32s/it]

...

Kept 477 axons.
Processing tile_masks_01737_02172.pkl
Loaded 404 masks.


 95%|█████████▌| 19/20 [01:05<00:03,  4.00s/it]

Kept 430 axons.
Processing tile_masks_01737_02635.pkl
Loaded 343 masks.


100%|██████████| 20/20 [01:08<00:00,  3.43s/it]

Kept 361 axons.


## Suppress overlapping objects

In [6]:
df = pd.concat(df_all_list, ignore_index=True)
df.label = df.index + 1
print(f'Starting number of axons: {len(df)}')

# move into the full image coordinate system
df['bbox-0'] = df['bbox-0'] + df['tile_row_start']
df['bbox-1'] = df['bbox-1'] + df['tile_col_start']
df['bbox-2'] = df['bbox-2'] + df['tile_row_start']
df['bbox-3'] = df['bbox-3'] + df['tile_col_start']
df['centroid-0'] = df['centroid-0'] + df['tile_row_start']
df['centroid-1'] = df['centroid-1'] + df['tile_col_start']

# remove not promissing objects based on updated parameters
df = df.loc[((df['euler_number'] < 1) & (df.sc_area/df.area_filled < 0.9)),:]

# remove edge objects of internal tile boundaries
df = mark_tile_edge_objects(df, pad=5, filter='not_edge')
print(f'Number of axons after removing edge objects: {len(df)}')
df['edge_ring'] = False

# supress by iou
df = df.reset_index(drop=True)
df_res = suppress_by_iou(df, iou_threshold=0.2)
df_res = df_res.loc[df_res.keep == 1,:]
print(f'Number of axons after suppressing overlaping objects: {len(df_res)}')

Starting number of axons: 7533
Number of axons after removing edge objects: 2022
Number of axons after suppressing overlaping objects: 548


## Find axons soma

In [24]:
inside_props = ['area', 'eccentricity', 'major_axis_length', 'minor_axis_length', 'centroid', 'bbox','image']
pad = 3

df_final = add_soma_data(df_res, inside_props, pad = pad)

# clean up labels
df_final = df_final.reset_index(drop=True)
df_final['label'] = df_final.index + 1
df_final['inside_label'] = df_final.index + 1


## Visualize cleaned masks

In [25]:
# Visualize myelin rings

mask = mask_from_df(df_final,im_shape, prefix='')
viewer.add_labels(mask, name='myelin rings', opacity = 0.8, scale = scale)

# Visualize detected axons

mask = mask_from_df(df_final, im_shape, prefix='inside_')
viewer.add_labels(mask, name='axons', opacity = 0.4, scale = scale)

<Labels layer 'axons' at 0x18b307a85d0>

## Save data frame

In [26]:
df_name = f'{prefix_save}axons.pkl'
df_final.to_pickle(os.path.join(output_dir,df_name))